<p float="left">
  <img src="climate_net/img/CCfCS.png" width="100" />
  <img src="climate_net/img/cam.png" width="100" />
</p>

# CCfCS ML for Climate Science -  Notebook 4. Afternoon Session
### Designing a machine learning algorithm for predicting extreme weather events in the ClimateNet dataset

The main goal of this afternoon's session is to use the ML tools from the morning session on the data that we initially explored to try and predict whether a given geographical location is experiencing an extreme weather event in the form of an atmospheric river or a tropical cyclone. The first thing to do is to create a training and testing dataset, which were preloaded in the MNIST example.

*You must have first added a shortcut to your GDrive - see [here](https://github.com/Ira-Shokar/CCFCS-Machine-Learning-Workshop):*

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%pip install -r /content/drive/MyDrive/climate_net/env.txt

In [ ]:
import numpy as np
import netCDF4 as nc
import matplotlib.pylab as plt
import cmocean 
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import urllib.request as urllib2
import requests
import xarray as xr
from scipy import stats
import tensorflow as tf
import tensorflow.keras as keras

Load in the data as usual

In [ ]:
ds = xr.open_dataset('/content/drive/MyDrive/climate_net/climatenet_data.nc')

In [ ]:
ds

Let us start by choosing some training inputs. You might want to go back to the data exploration notebook for a reminder of what these are, what they look like, and an idea of how they are related to each other. We also define our labels, which our network is trying to predict. Remember, 0 is no extreme event, 1 is an tropical cyclone and 2 is a atmospheric river. 

In [ ]:
input_1 = ds.U850.data
input_2 = ds.V850.data
input_3 = ds.TMQ.data
input_4 = ds.PSL.data
labels = ds.LABELS.data

To make the problem more tractable, let's sparse down the data a little by filtering it like we did earlier. Note this might take a little while to run.

*Note: is this the best way to filter the data? What happens to the performance of the model if we just sparse regularly, for example? These are questions you might come back to later if you have time. Convolutional networks are a special class of neural network that essentially optimise a filtering process to sequentially reduce dimensionaility whilst extracting relevant spatial information from images.*

In [ ]:
from scipy.ndimage.filters import gaussian_filter
filter_input_1 = gaussian_filter(input_1,sigma = [0,10,10])[:,::16,::16]
filter_input_2 = gaussian_filter(input_2,sigma = [0,10,10])[:,::16,::16]
filter_input_3 = gaussian_filter(input_3,sigma = [0,10,10])[:,::16,::16]
filter_input_4 = gaussian_filter(input_4,sigma = [0,10,10])[:,::16,::16]

In [ ]:
print(filter_input_1.shape)

Now we have 48 x 72 = 3456 inputs per time step. We filter the labels slightly differently, i.e. a filtered cell is labelled as an extreme event according to its Gaussian centre cell. 

In [ ]:
filter_labels = labels[:,::16,::16]

At this point the data is still in a nice form (i.e. we can do a contour plot for each timestep). Let us choose the first 67 time steps for training, and the remaining 16 for testing. We then need to convert it into the shape (training samples, number of inputs). 

In [ ]:
X_train = np.stack((filter_input_1[:67].flatten(), filter_input_2[:67].flatten(), 
                         filter_input_3[:67].flatten(), filter_input_4[:67].flatten()), axis=1)
X_test = np.stack((filter_input_1[67:].flatten(), filter_input_2[67:].flatten(), 
                         filter_input_3[67:].flatten(), filter_input_4[67:].flatten()), axis=1)
print(X_train.shape)
print(X_test.shape)

We need to similarly flatten the labels, and then convert them into the categorical format, just as we did for the MNIST labels.

In [ ]:
train_labels = filter_labels[:67].flatten()
test_labels = filter_labels[67:].flatten()
print(train_labels.shape)
print(test_labels.shape)
nb_classes= 3
Y_train = keras.utils.to_categorical(train_labels, num_classes=nb_classes, dtype='int64') 
Y_test = keras.utils.to_categorical(test_labels, num_classes=nb_classes, dtype='int64') 
print(Y_train.shape)
print(Y_test.shape)

The final step for preparing the training data is to normalize it. This is particularly important for our dataset since the range of values varies greatly for the different inputs (for example, PSL compared to TMQ). Here, we normalize by subtracting the mean and dividing by the standard deviation. Remember to normalize the test data by the same mean and standard deviation as the training data.

*You might want to experiment with different ways of normalizing the data, or look at what happens when there is no normalization.*

In [ ]:
mu = np.mean(X_train, axis=0)
sigma = np.std(X_train, axis=0)
print(mu, sigma)
X_train = (X_train-mu)/sigma #Careful not to run this cell twice 
X_test = (X_test-mu)/sigma

Now that the data is prepared, let's build our first model. This is identical to the MNIST model we built, except that the shape of the inputs and outputs has been changed to match our data.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[4, 1]),
    tf.keras.layers.Dense(200, activation="relu"),
    tf.keras.layers.Dense(60, activation="relu"),
    tf.keras.layers.Dense(3, activation='sigmoid')]) # classifying into 3 classes

In [ ]:
print(model.summary())

In [ ]:
model.compile(
    optimizer='adam',
    loss= 'categorical_crossentropy',
    metrics=['accuracy']) # % of correct answers

Train the model, just as we did for the MNIST data.

In [ ]:
model.fit(X_train, Y_train, batch_size=128, epochs=5, validation_split = 0.05)

We've trained our first model for making predictions on the data. Now let's have a look at how the model performs on the test set. We'll start by doing things qualitatively: reshaping the predicted classes back into their original spatio-temporal form and then plotting some time slices.

In [ ]:
predicted_classes = model.predict(X_test)
predicted_labels = np.argmax(predicted_classes, axis=1).reshape((16, 48, 72))

In [ ]:
plt.figure(figsize=(15,15))
for timestep in range(5):
    plt.subplot(5,2,2*timestep+1)
    plt.pcolormesh(predicted_labels[timestep])
    if timestep==0:
        plt.title('Predictions')
    plt.subplot(5,2,2*timestep+2)
    plt.pcolormesh(np.reshape(test_labels, (16,48,72))[timestep])
    if timestep==0:
        plt.title('True labels')
plt.show()

At least qualititavely, our models seems to be doing an OK job at predicting atmospheric rivers (in yellow) and a poor job at predicting tropical cyclones (in turquoise). Let's define and discuss some quantitative measures to expand on this. Firstly, accuracy: how often do predictions=labels?

In [ ]:
def accuracy(predicted, true):
    return np.sum(predicted==true)/predicted.shape[0]

In [ ]:
print(accuracy(predicted_labels.flatten(), test_labels.flatten()))

The model has very good accuracy on the test set. But how meaningful is this? Consider a baseline model that predicts class 0 for every grid cell. Then since most of the grid cells are indeed class 0, the accuracy will naturally be very high. Indeed, we can compute it:

In [ ]:
print(accuracy(np.zeros(test_labels.shape[0]), test_labels.flatten()))

Actually, our model only just exceeds the baseline by this metric. Ideally we would like a metric that captures how often the model is right when it predicts an extreme weather event. If we define a 'positive' to be a given class then we can define **precision** and **recall** as:

Precision = #True positives/(#True positives + #False positives) 

Recall = #True positives/(#True positives + #False negatives)

Here a true positive means the model correctly predicted true, and a false positive means the model incorrectly predicted true. If we want to seek a balance between precision and recall we can use the so-called F1 score:

F1 = 2 x (precision x recall)/(precision + recall)

For an extended discussion on the meaning of these metrics, see this article: https://towardsdatascience.com/accuracy-precision-recall-or-f1-331fb37c5cb9

In [ ]:
def precision_label_n(predicted, true, label_n):
    tp = ((predicted==label_n).astype(int)*(true==label_n).astype(int)).sum() #number of true positives
    fp = ((predicted==label_n).astype(int)*(1-(true==label_n).astype(int))).sum() #number of false positives
    return tp/(tp+fp)

In [ ]:
print('Cyclone precision', precision_label_n(predicted_labels.flatten(), test_labels.flatten(), label_n=1))
print('AR precision', precision_label_n(predicted_labels.flatten(), test_labels.flatten(), label_n=2))

In [ ]:
def recall_label_n(predicted, true, label_n):
    tp = ((predicted==label_n).astype(int)*(true==label_n).astype(int)).sum() #number of true positives
    fn = ((1-(predicted==label_n).astype(int))*((true==label_n).astype(int))).sum() #number of false negatives
    return tp/(tp+fn)

In [ ]:
print('Cyclone recall', recall_label_n(predicted_labels.flatten(), test_labels.flatten(), label_n=1))
print('Atmospheric river recall', recall_label_n(predicted_labels.flatten(), test_labels.flatten(), label_n=2))

There is actually a pre-existing library function that will do this for you, for each label. The 'support' is just the number of (true) labels from a given class. 

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(test_labels.flatten(), predicted_labels.flatten())

In [ ]:
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))

Here we can clearly see that the fact that the number of instances of each class is seriously skewed is causing some problems for the model. In particular, the recall scores are particularly poor for extreme weather events, i.e. there are a lot of false negatives. This is bad news: we would probably rather have an overcautious model rather than our current very undercautious one (i.e. fails to predict a lot of true extreme weather events). 

Of course, we should keep in mind that evaluating our model cell-by-cell is also probably not the best measure of performance: certainly the qualitative picture seems to suggest the regional performance on atmospheric rivers is actually OK: i.e. it gets the locations roughly correct (which cell-by-cell metrics of performance do not capture completely).

# The task
We have built a very basic model for predicting atmospheric weather events, and whilst it isn't great, there is an indication that it is picking up on some trends. By copying and adapting the code above, can you improve the model? 

Some ideas to get you started:

1. You could try getting started by simply training the model for more epochs. How much does this improve the performance metrics?
2. Next, you might try changing the model design. Why not play around with the number of layers, the number of neurons per layer, and the activation functions?
3. You might try changing the inputs to the model (look at the data loading notebook from this morning for some ideas). You could also try adding more inputs to the model, changing the filtering procedure, and changing the normalization.
4. **Extensions**: there is certainly spatial information in the original data that will be useful to a model for detecting extreme weather events. How might you harness this? One simple idea is to actually have latitude and/or longitude values of each grid cell each *as inputs to the model*. Another simple idea is to reduce the size of the dataset by filtering out the high latitudes. This will reduce the skew in the number of labels for each class. A more complex idea is to create a dataset which predicts a cell label based on data values from that cell and the adjacent cells (note this will require substantially reshaping your training data). 

Do feel free to discuss with each other and Sam/Seb/Ira if you aren't sure what to do! We do stress that the aim of this workshop is not to create a ground-breaking model for predicting extreme weather events, but rather to get experience constructing a dataset, and playing around with simple machine learning models for making predictions. 

### Choose inputs

In [ ]:
input_1 = ds.U850.data
input_2 = ds.V850.data
input_3 = ds.TMQ.data
input_4 = ds.PSL.data
# input_5 = ? 
labels = ds.LABELS.data

### Filter to reduce number of data points

In [ ]:
from scipy.ndimage.filters import gaussian_filter
filter_input_1 = gaussian_filter(input_1,sigma = [0,10,10])[:,::16,::16]
filter_input_2 = gaussian_filter(input_2,sigma = [0,10,10])[:,::16,::16]
filter_input_3 = gaussian_filter(input_3,sigma = [0,10,10])[:,::16,::16]
filter_input_4 = gaussian_filter(input_4,sigma = [0,10,10])[:,::16,::16]
filter_labels = labels[:,::16,::16]

### Build training and testing datasets

In [ ]:
X_train = np.stack((filter_input_1[:67].flatten(), filter_input_2[:67].flatten(), 
                         filter_input_3[:67].flatten(), filter_input_4[:67].flatten()), axis=1)
X_test = np.stack((filter_input_1[67:].flatten(), filter_input_2[67:].flatten(), 
                         filter_input_3[67:].flatten(), filter_input_4[67:].flatten()), axis=1)
print(X_train.shape)
print(X_test.shape)

In [ ]:
train_labels = filter_labels[:67].flatten()
test_labels = filter_labels[67:].flatten()
print(train_labels.shape)
print(test_labels.shape)
nb_classes= 3
Y_train = keras.utils.to_categorical(train_labels, num_classes=nb_classes, dtype='int64') 
Y_test = keras.utils.to_categorical(test_labels, num_classes=nb_classes, dtype='int64') 
print(Y_train.shape)
print(Y_test.shape)

### Normalize

In [ ]:
mu = np.mean(X_train, axis=0)
sigma = np.std(X_train, axis=0)
print(mu, sigma)
X_train = (X_train-mu)/sigma #Careful not to run this cell twice 
X_test = (X_test-mu)/sigma

### Build model

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[4, 1]),
    tf.keras.layers.Dense(200, activation="relu"),
    tf.keras.layers.Dense(60, activation="relu"),
    tf.keras.layers.Dense(3, activation='sigmoid')]) # classifying into 3 classes

In [ ]:
print(model.summary())

In [ ]:
model.compile(
    optimizer='adam',
    loss= 'categorical_crossentropy',
    metrics=['accuracy']) # % of correct answers

### Train model

In [ ]:
model.fit(X_train, Y_train, batch_size=128, epochs=10, validation_split = 0.05)

### Test model 

In [ ]:
predicted_classes = model.predict(X_test)
predicted_labels = np.argmax(predicted_classes, axis=1).reshape((16, 48, 72))

In [ ]:
plt.figure(figsize=(15,15))
for timestep in range(5):
    plt.subplot(5,2,2*timestep+1)
    plt.pcolormesh(predicted_labels[timestep])
    if timestep==0:
        plt.title('Predictions')
    plt.subplot(5,2,2*timestep+2)
    plt.pcolormesh(np.reshape(test_labels, (16,48,72))[timestep])
    if timestep==0:
        plt.title('True labels')
plt.show()

### Performance metrics

In [ ]:
from sklearn.metrics import precision_recall_fscore_support as score
precision, recall, fscore, support = score(test_labels.flatten(), predicted_labels.flatten())

In [ ]:
print('precision: {}'.format(precision))
print('recall: {}'.format(recall))
print('fscore: {}'.format(fscore))
print('support: {}'.format(support))